The goal of this notebook is to understand the original data structure and define basic operations.

The source data explored here has been previously downloaded from Path of Exile stash tabs API to local .txt files. Each file contained in a specific folder represent a single API call to http://www.pathofexile.com/api/public-stash-tabs?id= with a proper **nextChangeId**: this id is iteratively applyed on the next API call.

# TODO

1. add column to distinguish between unique and other items

In [8]:
import os
import re
import json

import pandas as pd

In [9]:
pd.set_option('display.max_columns', 500)

DATA_PATH = '../../data/stashes/'
stashes_path = os.listdir(DATA_PATH)

CURRENCY_KEY = ['alt', 'fuse', 'alch', 'chaos', 'gcp', 'exa', 'chrom', 'jew', 'chance', 'chisel', 'scour', 'blessed', 'regret', 'regal', 'divine', 'vaal', 'silver']
CURRENCY_NAME = ["Orb of Alteration", "Orb of Fusing", "Orb of Alchemy", "Chaos Orb", "Gemcutter's Prism", "Exalted Orb", "Chromatic Orb", "Jeweller's Orb", "Orb of Chance", "Cartographer's Chisel", "Orb of Scouring", "Blessed Orb", "Orb of Regret", "Regal Orb", "Divine Orb", "Vaal Orb", "Silver Coin"]

ITEMS_CATEGORIES = ['accessories', 'armour', 'jewels', 'weapons', 'currency']

In [10]:
def get_stashes_dict(path):
    with open(path, 'rb') as file:
        return json.load(file)

def get_stashes_items(verbose=False):
    values = list()
    
    for v in stashes_path:
        for stash in get_stashes_dict(DATA_PATH + v):
            
            if len(stash['items']) > 0 and stash['public']:
                # add stash related data to single items
                for item in stash['items']:
                    item['stash_note'] = stash['stash']
                    item['league'] = stash['league']

                values.extend(stash['items'])
            
    return values

def get_string_price(string):
    if re.match('(~price|~b/o)\s\d+((\.|/)\d+)?\s\w+', string) is not None:
        tokens = string.split(' ')
        
        currency, quantity = tokens[2], tokens[1]
        
        # check if currency and quantity are allowed values
        if currency in CURRENCY_KEY:
            return currency, quantity
    
    raise ValueError()
        

# Data exploration

In [11]:
items = get_stashes_items()

items = pd.DataFrame(items)

# alphabetically order dataframe columns
items = items[sorted(items.columns)]

items.shape

(432781, 54)

In [12]:
# delete useless data

deletable = ['abyssJewel', 'additionalProperties', 'artFilename', 'cisRaceReward', 'cosmeticMods', 'delve', 'descrText', 'flavourText', 'frameType', 'h', 'icon', 'id', 'inventoryId', 'maxStackSize', 'nextLevelRequirements', 'prophecyText', 'seaRaceReward', 'secDescrText', 'socketedItems', 'support', 'thRaceReward', 'utilityMods', "verified", "w", "x", "y"]

missing = list()
for v in deletable:
    try:
        del(items[v])
    except:
        missing.append(v)

if len(missing) > 0:
    print('Missing columns: {}'.format(missing))

In [13]:
# pandas built-in filling functionality
items.corrupted.fillna(False, inplace=True)
items['duplicated'].fillna(False, inplace=True)
items.elder.fillna(False, inplace=True)
items.fractured.fillna(False, inplace=True)
items.isRelic.fillna(False, inplace=True)
items.shaper.fillna(False, inplace=True)
items.synthesised.fillna(False, inplace=True)
items.note.fillna('', inplace=True)

In [14]:
# fill missing values with empty list
items['craftedMods'] = items.craftedMods.apply(lambda y: y if isinstance(y, list) else [])
items['enchantMods'] = items.enchantMods.apply(lambda y: y if isinstance(y, list) else [])
items['explicitMods'] = items.explicitMods.apply(lambda y: y if isinstance(y, list) else [])
items['implicitMods'] = items.implicitMods.apply(lambda y: y if isinstance(y, list) else [])
items['fracturedMods'] = items.fracturedMods.apply(lambda y: y if isinstance(y, list) else [])
items['veiledMods'] = items.veiledMods.apply(lambda y: y if isinstance(y, list) else [])
items['sockets'] = items.sockets.apply(lambda y: y if isinstance(y, list) else [])
items['properties'] = items.properties.apply(lambda y: y if isinstance(y, list) else [])
items['requirements'] = items.requirements.apply(lambda y: y if isinstance(y, list) else [])

In [15]:
# split items category by adding subcategory
items['subCategory'] = items.category.apply(lambda y: list(y.items())[0][1][0] if len(list(y.items())[0][1]) > 0 else None)
items['category'] = items.category.apply(lambda y: list(y.items())[0][0])

items.drop(index=items[~items.category.isin(ITEMS_CATEGORIES)].index, inplace=True)

In [16]:
# extract items price
price = []

for k, v in items.iterrows():
    try:
        # item has a valid price
        item_price = get_string_price(v.note)
    except:
        try:
            # stash has a valid price
            item_price = get_string_price(v.stash_note)
        except:
            # both item and stash have no valid price
            item_price = (None, None)
        
    price.append(item_price)

(items['priceCurrency'], items['priceQuantity']) = zip(*price)
    
# drop unpriced items
items.drop(index = items[(items.priceCurrency.isna()) & (items.priceQuantity.isna())].index, inplace=True)
del(items['note'], items['stash_note'])

## Currencies

In [17]:
currencies = items[items.category=='currency']

In [24]:
# delete useless data

deletable = ['category', 'corrupted', 'craftedMods', 'duplicated', 'elder', 'enchantMods', 'explicitMods', 'fractured', 'fracturedMods', 'identified', 'ilvl', 'implicitMods', 'isRelic', 'name', 'properties', 'requirements', 'shaper', 'sockets' ,'synthesised', 'subCategory', 'talismanTier', 'vaal', 'veiled', 'veiledMods']

missing = list()
for v in deletable:
    try:
        del(currencies[v])
    except:
        missing.append(v)

if len(missing) > 0:
    print('Missing columns: {}'.format(missing))

Missing columns: ['category', 'corrupted', 'craftedMods', 'duplicated', 'elder', 'enchantMods', 'explicitMods', 'fractured', 'fracturedMods', 'identified', 'ilvl', 'implicitMods', 'isRelic', 'name', 'properties', 'requirements', 'shaper', 'sockets', 'synthesised', 'subCategory', 'talismanTier', 'vaal', 'veiled']


In [22]:
currencies = currencies[currencies.typeLine.isin(CURRENCY_NAME)]

currencies['sellCurrency'] = currencies.typeLine.apply(lambda y: CURRENCY_KEY[CURRENCY_NAME.index(y)])

del(currencies['typeLine'])

AttributeError: 'DataFrame' object has no attribute 'typeLine'

In [25]:
currencies.rename(columns={"stackSize": "sellQuantity"}, inplace=True)

currencies.sample()

,league,sellQuantity,priceCurrency,priceQuantity,sellCurrency
361413,Synthesis,12.0,chaos,10,chrom


## Mod based items

In [26]:
def items_mods_formatting(items, target_mod):

    witems_mods_vocabulary = []
    witems_mods = []

    for k, v in items.iterrows():
        for mod in v[target_mod]:
            item_mod = dict()
            item_mod['itemId'] = k

            generic_mod = re.sub('\d+', '##', mod)
            try:
                mod_index = witems_mods_vocabulary.index(generic_mod)
            except:
                witems_mods_vocabulary.append(generic_mod)
                mod_index = len(witems_mods_vocabulary) - 1

            item_mod['modId'] = mod_index

            for i, v in enumerate(re.findall('\d+', mod)):
                item_mod['value{}'.format(i)] = v

            witems_mods.append(item_mod)

    witems_mods = pd.DataFrame(witems_mods)
    witems_mods_vocabulary = pd.DataFrame(witems_mods_vocabulary, columns=['text'])
    
    return witems_mods, witems_mods_vocabulary

### Categories based analysis

In [27]:
mod_items = items[items.category.isin(['armour', 'accessories', 'weapons', 'jewels']) ]

temp_mod_items = mod_items.copy()
temp_mod_items['craftedMods'] = mod_items.craftedMods.apply(lambda y : [re.sub('\d+(.\d+)?', 'N', v) for v in y])
temp_mod_items['enchantMods'] = mod_items.enchantMods.apply(lambda y : [re.sub('\d+(.\d+)?', 'N', v) for v in y])
temp_mod_items['explicitMods'] = mod_items.explicitMods.apply(lambda y : [re.sub('\d+(.\d+)?', 'N', v) for v in y]).apply(lambda y: [v.split(':')[1].strip() if ':' in v else v for v in y])
temp_mod_items['fracturedMods'] = mod_items.fracturedMods.apply(lambda y : [re.sub('\d+(.\d+)?', 'N', v) for v in y])
temp_mod_items['implicitMods'] = mod_items.implicitMods.apply(lambda y : [re.sub('\d+(.\d+)?', 'N', v) for v in y])

In [28]:
crafted_s = set([v for l in temp_mod_items.craftedMods.values for v in l])
enchant_s = set([v for l in temp_mod_items.enchantMods.values for v in l])
explicit_s = set([v for l in temp_mod_items.explicitMods.values for v in l])
fractured_s = set([v for l in temp_mod_items.fracturedMods.values for v in l])
implicit_s = set([v for l in temp_mod_items.implicitMods.values for v in l])
print(len(crafted_s), len(enchant_s), len(explicit_s), len(fractured_s), len(implicit_s))

# mods sets intersections
print(len(crafted_s & enchant_s), len(crafted_s & explicit_s), len(crafted_s & fractured_s), len(crafted_s & implicit_s))
print(len(enchant_s & explicit_s), len(enchant_s & fractured_s), len(enchant_s & implicit_s))
print(len(explicit_s & fractured_s), len(explicit_s & implicit_s))
print(len(fractured_s & implicit_s))

183 495 2092 149 287
1 139 77 86
8 0 0
149 222
88


In [29]:
armour = temp_mod_items[temp_mod_items.category=='armour']
accessories = temp_mod_items[temp_mod_items.category=='accessories']
jewels = temp_mod_items[temp_mod_items.category=='jewels']
weapons = temp_mod_items[temp_mod_items.category=='weapons']

In [30]:
armour_mods_s = set([v for l in armour.implicitMods.values for v in l])
accessories_mods_s = set([v for l in accessories.implicitMods.values for v in l])
jewels_mods_s = set([v for l in jewels.implicitMods.values for v in l])
weapons_mods_s = set([v for l in weapons.implicitMods.values for v in l])
print(len(armour_mods_s), len(accessories_mods_s), len(jewels_mods_s), len(weapons_mods_s))

# intersection between mods sets
print(len(armour_mods_s & accessories_mods_s), len(armour_mods_s & jewels_mods_s), len(armour_mods_s & weapons_mods_s))
print(len(accessories_mods_s & jewels_mods_s), len(accessories_mods_s & weapons_mods_s))
print(len(jewels_mods_s & weapons_mods_s))

141 123 26 93
54 6 28
7 26
4


### Wearable items

In [28]:
witems = items[items.category.isin(['armour', 'weapons', 'accessories'])].copy()

In [29]:
# split veiledMods into a much intuitive data structure
witems['veiledPrefix'] = witems.veiledMods.apply(lambda y: sum(['Prefix' in v for v in y]))
witems['veiledSuffix'] = witems.veiledMods.apply(lambda y: sum(['Suffix' in v for v in y]))

del(witems['veiled'])
del(witems['veiledMods'])

In [31]:
witems_base_quality = list()
for k, v in witems.iterrows():
    witem_quality = 0
    for i, prop in enumerate(v.properties):
        if prop['name'] == 'Quality':            
#             witem_quality = int(v.properties.pop(i)['values'][0][0][1:-1])
            witem_quality = int(prop['values'][0][0][1:-1])
            del(v.properties[i])
            
    for mod in v.craftedMods:
        if re.match(r'^\+\d+%\sto\sQuality$', mod):
            witem_quality -= int(mod.split(' ')[0][1:-1])
    
    witems_base_quality.append(witem_quality)
    
witems['baseQuality'] = witems_base_quality

In [32]:
# remove items useless properties such as item name (all the properties which don't have a defined value)
witems['properties'] = pd.Series([pname for iprop in witems.properties.apply(lambda y: [v['name'] for v in y]) for pname in iprop]).value_counts()

In [96]:
witems[witems.properties.notna()].properties

Series([], Name: properties, dtype: float64)

In [39]:
# extract items requirements
itemLvlRequirement, itemDexRequirement, itemStrRequirement, itemIntRequirement = [], [], [], []

for k, v in witems.iterrows():
    iLvlReq, iStrReq, iDexReq, iIntReq = 0, 0, 0, 0
    for ireq in v['requirements']:
        req_value = int(ireq['values'][0][0])
        
        if ireq['name'] == 'Level':
            iLvlReq = req_value
        elif ireq['name'] in ['Str', 'Strength']:
            iStrReq = req_value
        elif ireq['name'] in ['Dex', 'Dexterity']:
            iDexReq = req_value
        elif ireq['name'] in ['Int', 'Intelligence']:
            iIntReq = req_value

    itemLvlRequirement.append(iLvlReq)
    itemDexRequirement.append(iStrReq)
    itemStrRequirement.append(iDexReq)
    itemIntRequirement.append(iIntReq)
        
witems['lvlRequirement'] = itemLvlRequirement
witems['dexRequirement'] = itemDexRequirement
witems['strRequirement'] = itemStrRequirement
witems['intRequirement'] = itemIntRequirement

In [35]:
# extract items sockets into a separate dataframe
witems_sockets = list()
for k, v in witems.iterrows():
    for socket in v['sockets']:
        witem_socket = dict()
        witem_socket['itemId'] = k
        witem_socket['group'] = socket['group']
        witem_socket['socketColour'] = socket['sColour']
        
        witems_sockets.append(witem_socket)
        
witems_sockets = pd.DataFrame(witems_sockets)

In [36]:
witems_crafted_mods, crafted_mods = items_mods_formatting(witems, 'craftedMods')
witems_enchant_mods, enchant_mods = items_mods_formatting(witems, 'enchantMods')
witems_explicit_mods, explicit_mods = items_mods_formatting(witems, 'explicitMods')
witems_fractured_mods, fractured_mods = items_mods_formatting(witems, 'fracturedMods')
witems_implicit_mods, implicit_mods = items_mods_formatting(witems, 'implicitMods')

In [43]:
deletable = ['name', 'properties', 'requirements', 'sockets', 'stackSize', 'typeLine', 'vaal', 'craftedMods', 'enchantMods', 'explicitMods', 'fracturedMods', 'implicitMods']

missing = list()
for v in deletable:
    try:
        del(witems[v])
    except:
        missing.append(v)

if len(missing) > 0:
    print('Missing columns: {}'.format(missing))

Missing columns: ['name', 'requirements', 'sockets', 'stackSize', 'typeLine', 'vaal', 'craftedMods', 'enchantMods', 'explicitMods', 'fracturedMods', 'implicitMods']


In [44]:
witems.sample()

,category,corrupted,duplicated,elder,fractured,identified,ilvl,isRelic,league,shaper,synthesised,talismanTier,subCategory,priceCurrency,priceQuantity,veiledPrefix,veiledSuffix,baseQuality,lvlRequirement,dexRequirement,strRequirement,intRequirement
265043,accessories,False,False,False,False,True,74,False,Standard,False,False,NaN,belt,chisel,6,0,0,0,22,0,0,0


### Jewels

In [12]:
jewels = items[items.category.isin(['jewels'])].copy()

In [13]:
jewels.columns

Index(['category', 'corrupted', 'craftedMods', 'duplicated', 'elder',
       'enchantMods', 'explicitMods', 'fractured', 'fracturedMods',
       'identified', 'ilvl', 'implicitMods', 'isRelic', 'league', 'name',
       'properties', 'requirements', 'shaper', 'sockets', 'stackSize',
       'synthesised', 'talismanTier', 'typeLine', 'vaal', 'subCategory',
       'veiledPrefix', 'veiledSuffix', 'priceCurrency', 'priceQuantity'],
      dtype='object')

In [14]:
jewels_radius = list()

for k, v in jewels['properties'].iteritems():
    if not isinstance(v, list):
        jewels_radius.append('None')
        
    else:
        j_rad = None
        for j_prop in v:
            if j_prop['name'] == 'Radius':
                j_rad = j_prop['values'][0][0]
        
        jewels_radius.append(j_rad if j_rad is not None else 'None')
    
jewels['radius'] = jewels_radius

In [15]:
jewels['lvlRequirement'] = jewels['requirements'].apply(lambda y: y[0]['values'][0][0] if isinstance(y, list) else 0).value_counts()

In [16]:
jewels['isAbyss'] = jewels['subCategory'].apply(lambda y: True if y == 'abyss' else False)

In [19]:
deletable = ['category', 'craftedMods', 'elder', 'enchantMods', 'name',
        'properties', 'requirements', 'shaper', 'sockets', 'stackSize', 'subCategory', 
        'talismanTier', 'typeLine', 'vaal', 'veiledPrefix', 'veiledSuffix']

missing = list()
for v in deletable:
    try:
        del(jewels[v])
    except:
        missing.append(v)

if len(missing) > 0:
    print('Missing columns: {}'.format(missing))

Missing columns: ['category', 'craftedMods', 'elder', 'enchantMods', 'name', 'properties', 'requirements', 'shaper', 'sockets', 'stackSize', 'talismanTier', 'typeLine', 'vaal', 'veiledPrefix', 'veiledSuffix']


In [20]:
jewels.sample()

,corrupted,duplicated,explicitMods,fractured,fracturedMods,identified,ilvl,implicitMods,isRelic,league,synthesised,priceCurrency,priceQuantity,radius,lvlRequired,isAbyss
40982,False,False,[+1% Chance to Block Spell Damage while wieldi...,False,[],True,82,[],False,Standard,False,exa,1,None,NaN,False


In [56]:
jewels_mods = set([mod for v in jewels[jewels.implicitMods.apply(lambda y: len(y) > 0)].implicitMods.apply(lambda y: [re.sub('\d+(.\d+)?', 'N', v) for v in y]).values for mod in v])
len(jewels_mods)

26

In [57]:
jewels_mods = set([mod for v in jewels[jewels.explicitMods.apply(lambda y: len(y) > 0)].explicitMods.apply(lambda y: [re.sub('\d+(.\d+)?', 'N', v) for v in y]).values for mod in v])
len(jewels_mods)

529

In [58]:
jewels_mods = set([mod for v in jewels[jewels.fracturedMods.apply(lambda y: len(y) > 0)].fracturedMods.apply(lambda y: [re.sub('\d+(.\d+)?', 'N', v) for v in y]).values for mod in v])
len(jewels_mods)

75